In [ ]:
# Célula 1: Título do Projeto e Introdução

# Título do Projeto: Sistema de Gerenciamento de Bens Patrimoniais com MongoDB
# Disciplina: Banco de Dados Não Relacional - UTFPR
# Professor(a): Kelly Lais Wiggers
# Aluno(a): João Abner Pereira de Almeida
# Data de Entrega: 26/06/2025

"""
Objetivo do Projeto:
Este projeto final tem como objetivo desenvolver um sistema de armazenamento 
e consulta de dados patrimoniais utilizando MongoDB e Python.

Ele simula o backend de um aplicativo de gestão de patrimônio, permitindo:

1.  Armazenar dados de bens patrimoniais (com base em um dataset público).
2.  Registrar movimentações (simulando a leitura de códigos de barras com 'asset_id').
3.  Realizar operações CRUD (Criar, Ler, Atualizar, Excluir) exigidas pela disciplina.
4.  Executar consultas com diferentes operadores e funções de agregação.
5.  Realizar análise exploratória de dados com Pandas.
6.  Gerar relatórios visuais simples.

Este sistema é relevante para um aplicativo de Gestão de Patrimônio da UTFPR,
focado em substituir planilhas manuais, reduzir erros e agilizar inventários.
"""

print("Iniciando a demonstração do Projeto Final de Banco de Dados Não Relacional...")

In [ ]:
# Célula 2: Importações de Bibliotecas e Configurações Iniciais

import pandas as pd # para manipulação e análise de dados em Python
from pymongo import MongoClient # para interagir com o MongoDB
import json 
import os # para manipulação de arquivos no sistema operacional
import matplotlib.pyplot as plt # para criação de gráficos em Python.
import seaborn as sns # para visualização de dados estatísticos
from IPython.display import display # Adiciona importação para display

# --- CONFIGURAÇÃO DO MONGO DB ---
MONGO_URI = "mongodb://localhost:27017/"
DB_NAME = "patrimonio_db"
COLLECTION_NAME = "bens"

# Caminho para o arquivo CSV do dataset "Universal Product Code Database" do Kaggle. 
CSV_FILE_PATH = "products.csv"

print("Bibliotecas importadas e configurações iniciais definidas.")
print(f"MongoDB URI: {MONGO_URI}")
print(f"Banco de Dados: {DB_NAME}, Coleção: {COLLECTION_NAME}")
print(f"Caminho do CSV: {CSV_FILE_PATH}")

In [ ]:
# Célula 3: Função de Conexão com MongoDB (Item 2)

def conectar_mongodb():
    """
    Estabelece a conexão com o MongoDB.
    Retorna o objeto da coleção para interação ou None em caso de erro.

    Item 2: Define a forma de conexão e acesso aos dados no MongoDB.
    """
    try:
        client = MongoClient(MONGO_URI)
        db = client[DB_NAME]
        collection = db[COLLECTION_NAME]
        # Tentativa de comando 'ping' para verificar se a conexão está ativa.
        client.admin.command('ping')
        print(f"Conectado com sucesso ao MongoDB: Banco '{DB_NAME}', Coleção '{COLLECTION_NAME}'.")
        return collection
    except Exception as e:
        print(f"ERRO: Não foi possível conectar ao MongoDB. Verifique se o servidor MongoDB está em execução em '{MONGO_URI}'.")
        print(f"Detalhes do erro: {e}")
        return None

# Tentando conectar ao MongoDB
bens_collection = conectar_mongodb()

In [ ]:
# Célula 4: Função de Importação de Dados do CSV para MongoDB (Itens 2a, 2b, 2c)

# A base de dados pública que escolhi para o projeto é a 
#'Universal Product Code Database' do Kaggle.

def importar_csv_para_mongodb(collection, csv_path, num_registros=100):
    """
    Importa dados de um arquivo CSV para a coleção MongoDB.
    Limita o número de registros para agilizar o projeto.
    Mapeia campos do dataset original para o modelo de bens patrimoniais.

    Item 2a: Utiliza base de dados pública (products.csv do Kaggle).
    Item 2b: Converte dados para formato JSON compatível com MongoDB.
    Item 2c: Importa os dados para manipulação no Jupyter Notebook (via MongoDB).
    """
    if not os.path.exists(csv_path):
        print(f"ERRO: Arquivo CSV '{csv_path}' não encontrado. Por favor, verifique o caminho.")
        return

    try:
        # Lê o CSV com pandas
        df = pd.read_csv(csv_path, low_memory=False) # low_memory=False para evitar avisos e erros de leitura do pandas
        
        # Verifica se o DataFrame está vazio
        print(f"CSV '{csv_path}' carregado. Total de registros no arquivo: {len(df)}.")

        # Seleciona uma amostra para o projeto (ajustável).
        # Se o DataFrame tiver menos registros que num_registros, usa todo o DataFrame.
        df_sample = df.head(num_registros) if len(df) > num_registros else df 
        print(f"Processando e importando {len(df_sample)} registros para o MongoDB.")
        
        # Mapeia os campos do DataFrame para o formato de documento MongoDB
        documents = []
        for index, row in df_sample.iterrows():
            doc = {
                "asset_id": str(row.get("product_id", f"UNKNOWN_ID_{index}")), # product_id como asset_id
                "asset_name": str(row.get("product_name", "Nome Desconhecido")),
                "category": str(row.get("category", "Geral")),
                "brand": str(row.get("brand", "Não Informada")),
                "description": str(row.get("description", "Sem descrição.")),
                "location": "Armazenagem Principal", # Valor inicial
                "status": "Em Uso", # Valor inicial
                "acquisition_date": pd.to_datetime('2023-01-01').strftime('%Y-%m-%d'),
                "movement_history": [
                    {
                        "date": pd.to_datetime('2023-01-01 00:00:00').strftime('%Y-%m-%d %H:%M:%S'),
                        "type": "Aquisição",
                        "from_location": "Fornecedor",
                        "to_location": "Armazenagem Principal",
                        "responsible": "Sistema Inicial"
                    }
                ]
            }
            documents.append(doc)

        # Limpa a coleção para evitar duplicatas em cada execução e insere os novos documentos.
        collection.delete_many({})
        if documents:
            result = collection.insert_many(documents)
            print(f"{len(result.inserted_ids)} documentos importados com sucesso para o MongoDB na coleção '{COLLECTION_NAME}'.")
        else:
            print("Nenhum documento válido para importar.")

    except Exception as e:
        print(f"ERRO ao importar CSV para MongoDB: {e}")

# Executa a importação dos dados se a conexão for bem-sucedida
if bens_collection is not None:
    importar_csv_para_mongodb(bens_collection, CSV_FILE_PATH, num_registros=100)

In [ ]:
# Célula 5: Função para Apresentar Dados como DataFrame (Item 2d)

from IPython.display import display

def apresentar_dados_como_dataframe(collection, limit=10):
    """
    Busca documentos da coleção MongoDB e os apresenta como um DataFrame do Pandas.
    Exibe a coluna movement_history de forma mais legível.
    Item 2d: Apresenta os dados como DataFrame.
    """
    try:
        cursor = collection.find().limit(limit)
        data = list(cursor)

        if data:
            df = pd.DataFrame(data)
            if '_id' in df.columns:
                df = df.drop(columns=['_id']) # Remove o ID interno do MongoDB para melhor visualização
            
            # Formata a coluna movement_history para exibição mais amigável
            if 'movement_history' in df.columns:
                def format_history(hist):
                    if isinstance(hist, list) and len(hist) > 0: # Verifica se é uma lista e não está vazia
                        last = hist[-1] # Pega o último movimento
                        return f"{last.get('date', '')} | {last.get('type', '')} | {last.get('from_location', '')} → {last.get('to_location', '')}"
                    return "" # Retorna vazio se não houver histórico ou se for inválido
                
                # Aplica a formatação na coluna movement_history
                df['Última Movimentação'] = df['movement_history'].apply(format_history)
    
            pd.set_option('display.max_colwidth', 120) #
            pd.set_option('display.width', 1000) 
            # Exibe os primeiros registros como DataFrame
            print(f"\n--- Primeiros {len(df)} documentos da coleção '{COLLECTION_NAME}' como DataFrame ---")
            display(df)
            return df
        else:
            print(f"Nenhum documento encontrado na coleção '{COLLECTION_NAME}'.")
            return pd.DataFrame()
    except Exception as e:
        print(f"ERRO ao apresentar dados como DataFrame: {e}")
        return pd.DataFrame()

# Apresenta uma amostra dos dados recém-importados como DataFrame
if bens_collection is not None:
    df_patrimonio = apresentar_dados_como_dataframe(bens_collection, limit=15) # Exibe mais para demonstração

In [ ]:
# Célula 6: Demonstração de Inserção de Dados (Item 3b)

# Adiciona importação para display
from IPython.display import display 

def inserir_documento_extra(collection, num_docs=5):
    """
    Insere 'num_docs' documentos de dados extras na coleção, simulando novos bens.
    Item 3b: Em cada coleção, insira 5 documentos de dados extras.
    """
    new_assets = [] # Lista para armazenar os novos documentos
    current_count = collection.count_documents({}) # Conta o número atual de documentos na coleção
    
    # Itera para criar 5 novos documentos
    for i in range(num_docs): 
        asset_id_new = f"NEWASSET{current_count + 1 + i}" # Gera um novo asset_id único
        
        # Cria um novo documento com dados fictícios
        new_asset = {
            "asset_id": asset_id_new,
            "asset_name": f"Monitor 27 polegadas {i+1}",
            "category": "Eletrônico",
            "brand": "TechBrand",
            "description": f"Monitor de alta resolução para estação de trabalho {i+1}.",
            "location": "Sala de Reuniões",
            "status": "Ativo",
            "acquisition_date": pd.to_datetime('2024-05-20').strftime('%Y-%m-%d'),
            "movement_history": [
                {
                    "date": pd.to_datetime('2024-05-20 09:30:00').strftime('%Y-%m-%d %H:%M:%S'),
                    "type": "Aquisição",
                    "from_location": "Fornecedor",
                    "to_location": "Sala de Reuniões",
                    "responsible": "Gerência de TI"
                }
            ]
        }
        new_assets.append(new_asset) # Adiciona o novo documento à lista
    try:
        # Insere os novos documentos na coleção
        result = collection.insert_many(new_assets)
        print(f"\nINSERÇÃO: Inseridos {len(result.inserted_ids)} documentos extras na coleção.")
        # Exibe os novos documentos inseridos de forma resumida
        display(pd.DataFrame(new_assets))
        # Retorna os IDs dos documentos inseridos
        return result.inserted_ids 
    except Exception as e: # Captura qualquer erro durante a inserção
        print(f"ERRO ao inserir documentos extras: {e}")
        return []

# Executa a inserção de 5 documentos extras
if bens_collection is not None:
    inserted_ids_extras = inserir_documento_extra(bens_collection, num_docs=5)

In [ ]:
# Célula 7: Demonstração de Edição de Dados (Item 3c)
from IPython.display import display

def editar_documento(collection, asset_id, new_location=None, new_status=None):
    """
    Edita a localização ou status de um bem e registra a movimentação no histórico.
    """
    # Busca o bem pelo asset_id
    bem = collection.find_one({"asset_id": asset_id})
    if not bem:
        print(f"Bem '{asset_id}' não encontrado.")
        return

    update = {} # Dicionário para armazenar as atualizações
    historico = {
        "date": pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
        "responsible": "Usuário App"
    }

    # Solicita novas informações ao usuário
    if new_location and new_location != bem.get("location"): # Verifica se a nova localização é diferente da atual
        update["location"] = new_location
        historico["type"] = "Mudança de Localização"
        historico["from_location"] = bem.get("location", "Desconhecida")
        historico["to_location"] = new_location
        print(f"Localização alterada para '{new_location}'.")

    if new_status and new_status != bem.get("status"): 
        update["status"] = new_status
        historico["type"] = historico.get("type", f"Atualização de Status: {bem.get('status')} → {new_status}")
        print(f"Status alterado para '{new_status}'.")

    if update: # Se houver atualizações 
        # Atualiza o documento no MongoDB e adiciona o histórico de movimentação
        collection.update_one(
            {"asset_id": asset_id},
            {"$set": update, "$push": {"movement_history": historico}}
        )
        doc = collection.find_one({"asset_id": asset_id})
        if doc:
            df = pd.DataFrame([doc]) # Cria um DataFrame para exibir o documento atualizado
            if '_id' in df.columns: # Verifica se a coluna '_id' existe
                df = df.drop(columns=['_id']) # Remove o ID interno do MongoDB para melhor visualização
            display(df)
        print("Bem atualizado com sucesso!")
    else:
        print("Nenhuma alteração realizada.")

# Exemplo de uso:
if bens_collection is not None:
    # Edita um bem do dataset original
    sample = bens_collection.find_one({"asset_name": {"$ne": "Monitor 27 polegadas 1"}})
    if sample:
        editar_documento(bens_collection, sample['asset_id'], new_location="Reitoria - Sala A", new_status="Em Manutenção")
    # Edita um bem recém-inserido
    if inserted_ids_extras:
        doc = bens_collection.find_one({"_id": inserted_ids_extras[0]})
        if doc:
            editar_documento(bens_collection, doc['asset_id'], new_status="Desativado")

In [ ]:
# Célula 8: Demonstração de Busca por Todos os Documentos (Item 3d)

def buscar_todos_documentos(collection):
    """
    Busca e retorna todos os documentos de uma coleção.
    Item 3d: Permite realizar a busca por todos os documentos de cada coleção.
    """
    print(f"\n--- BUSCA: Todos os documentos na coleção '{COLLECTION_NAME}' ---")
    try:
        documents = list(collection.find({}))
        if documents:
            for doc in documents:
                print(doc)
            print(f"Total de documentos: {len(documents)}")
            return documents
        else:
            print("Nenhum documento encontrado na coleção.")
            return []
    except Exception as e:
        print(f"ERRO ao buscar todos os documentos: {e}")
        return []

# Executa a busca por todos os documentos
if bens_collection is not None:
    buscar_todos_documentos(bens_collection)

In [ ]:
# Célula 9: Demonstração de Exclusão de Dados (Item 3e)

def excluir_documento(collection, asset_id):
    """
    Exclui um documento da coleção pelo seu asset_id.
    Item 3e: Exclui 2 documentos em cada coleção.
    """
    try:
        result = collection.delete_one({"asset_id": asset_id})
        if result.deleted_count > 0:
            print(f"\nEXCLUSÃO: Documento com asset_id '{asset_id}' excluído com sucesso.")
        else:
            print(f"\nEXCLUSÃO: Documento com asset_id '{asset_id}' não encontrado para exclusão.")
    except Exception as e:
        print(f"ERRO ao excluir documento: {e}")

# Demonstração de exclusão (excluir os 2 primeiros documentos extras inseridos)
if bens_collection is not None:
    if inserted_ids_extras and len(inserted_ids_extras) >= 2:
        print("\n--- EXCLUINDO: 2 documentos extras recém-inseridos ---")
        # Pega o asset_id dos documentos pelos _ids inseridos
        asset_to_delete_1_doc = bens_collection.find_one({"_id": inserted_ids_extras[0]}, {"asset_id": 1})
        if asset_to_delete_1_doc:
            excluir_documento(bens_collection, asset_to_delete_1_doc['asset_id'])
        else:
            print("AVISO: Não foi possível encontrar o primeiro documento extra para exclusão.")

        if len(inserted_ids_extras) > 1:
            asset_to_delete_2_doc = bens_collection.find_one({"_id": inserted_ids_extras[1]}, {"asset_id": 1})
            if asset_to_delete_2_doc:
                excluir_documento(bens_collection, asset_to_delete_2_doc['asset_id'])
            else:
                print("AVISO: Não foi possível encontrar o segundo documento extra para exclusão.")
    else:
        print("\nAVISO: Não há documentos extras suficientes para demonstrar a exclusão de 2 documentos.")

# Opcional: Para verificar se as exclusões ocorreram (mostrar lista atualizada)
if bens_collection is not None:
    print("\n--- BUSCA: Documentos após as exclusões ---")
    buscar_todos_documentos(bens_collection)

In [ ]:
# Célula 10: Demonstração de Buscas com Operadores (Item 4)

# Item 4: Permita o uso de pelo menos 2 buscas utilizando diferentes operadores.

def buscar_bens_por_localizacao(collection, location):
    """
    Busca bens em uma localização específica. (Operador implícito $eq)
    """
    print(f"\n--- BUSCA: Bens na localização: '{location}' ---")
    try:
        assets = list(collection.find({"location": location})) # Usando $eq implícito
        if assets:
            for asset in assets:
                print(f"  ID: {asset['asset_id']}, Nome: {asset['asset_name']}, Status: {asset['status']}, Categoria: {asset['category']}")
            print(f"Total de bens encontrados: {len(assets)}")
            return assets
        else:
            print(f"Nenhum bem encontrado na localização '{location}'.")
            return []
    except Exception as e:
        print(f"ERRO ao buscar bens por localização: {e}")
        return []

def buscar_bens_com_status_ou_categoria(collection, status=None, category=None):
    """
    Busca bens com um determinado status OU uma determinada categoria. (Operador $or)
    """
    query = {}
    if status and category:
        query = {"$or": [{"status": status}, {"category": category}]}
        print(f"\n--- BUSCA: Bens com status '{status}' OU categoria '{category}' ---")
    elif status:
        query = {"status": status}
        print(f"\n--- BUSCA: Bens com status '{status}' ---")
    elif category:
        query = {"category": category}
        print(f"\n--- BUSCA: Bens com categoria '{category}' ---")
    else:
        print("Forneça um status ou uma categoria para a busca.")
        return []

    try:
        assets = list(collection.find(query))
        if assets:
            for asset in assets:
                print(f"  ID: {asset['asset_id']}, Nome: {asset['asset_name']}, Localização: {asset['location']}, Status: {asset['status']}, Categoria: {asset['category']}")
            print(f"Total de bens encontrados: {len(assets)}")
            return assets
        else:
            print("Nenhum bem encontrado com os critérios especificados.")
            return []
    except Exception as e:
        print(f"ERRO ao buscar bens por status ou categoria: {e}")
        return []

def buscar_historico_de_movimentacao(collection, asset_id):
    """
    Busca o histórico de movimentação de um bem específico.
    """
    print(f"\n--- BUSCA: Histórico de Movimentação para o Bem: '{asset_id}' ---")
    try:
        asset = collection.find_one({"asset_id": asset_id}, {"asset_name": 1, "movement_history": 1})
        if asset and "movement_history" in asset:
            print(f"Nome do Bem: {asset.get('asset_name', 'Não informado')}")
            if asset["movement_history"]:
                for entry in asset["movement_history"]:
                    print(f"  - Data: {entry.get('date')}, Tipo: {entry.get('type')}, De: {entry.get('from_location')}, Para: {entry.get('to_location')}, Responsável: {entry.get('responsible')}")
            else:
                print("  Nenhum histórico de movimentação para este bem.")
            return asset["movement_history"]
        else:
            print(f"Bem '{asset_id}' não encontrado ou sem histórico de movimentação.")
            return []
    except Exception as e:
        print(f"ERRO ao buscar histórico de movimentação: {e}")
        return []

# Executa as buscas
if bens_collection is not None:
    # Busca 1: por localização específica
    buscar_bens_por_localizacao(bens_collection, "Armazenagem Principal") # Deve encontrar muitos

    # Busca 2: por status ou categoria
    buscar_bens_com_status_ou_categoria(bens_collection, status="Em Manutenção", category="Eletrônico")
    buscar_bens_com_status_ou_categoria(bens_collection, status="Desativado") # Apenas por status

    # Busca 3: Histórico de um bem específico (pegando o asset_id do item editado anteriormente)
    # Tenta pegar o asset_id do item editado para mostrar o histórico
    if 'sample_asset_1' in locals() and sample_asset_1:
        buscar_historico_de_movimentacao(bens_collection, sample_asset_1['asset_id'])
    elif bens_collection.count_documents({}) > 0:
        # Se o sample_asset_1 não foi encontrado ou editado, tenta pegar um asset_id qualquer
        random_asset = bens_collection.find_one({}, {"asset_id": 1})
        if random_asset:
            buscar_historico_de_movimentacao(bens_collection, random_asset['asset_id'])
        else:
            print("\nAVISO: Não há bens no banco de dados para demonstrar histórico de movimentação.")
    else:
        print("\nAVISO: Não há bens no banco de dados para demonstrar histórico de movimentação.")

In [ ]:
# Célula 11: Demonstração de Funções de Agregação (Item 5)

# Item 5: Permita o uso de pelo menos 2 buscas utilizando diferentes funções de agregação.

def contar_bens_por_localizacao(collection):
    """
    Conta o número de bens por localização usando agregação ($group, $sum).
    """
    print("\n--- AGREGAÇÃO: Contagem de Bens por Localização ---")
    try:
        pipeline = [
            {"$group": {"_id": "$location", "count": {"$sum": 1}}}, # Agrupa por localização e conta
            {"$sort": {"count": -1}} # Ordena do maior para o menor
        ]
        results = list(collection.aggregate(pipeline))
        if results:
            for result in results:
                print(f"  Localização: {result['_id']}, Quantidade: {result['count']}")
            return results
        else:
            print("Nenhuma contagem por localização encontrada.")
            return []
    except Exception as e:
        print(f"ERRO ao contar bens por localização: {e}")
        return []

def contar_bens_por_categoria_e_status(collection):
    """
    Conta o número de bens por categoria e status combinados usando agregação ($group).
    """
    print("\n--- AGREGAÇÃO: Contagem de Bens por Categoria e Status ---")
    try:
        pipeline = [
            {"$group": {"_id": {"category": "$category", "status": "$status"}, "count": {"$sum": 1}}},
            {"$sort": {"_id.category": 1, "count": -1}} # Ordena por categoria e depois por contagem
        ]
        results = list(collection.aggregate(pipeline))
        if results:
            for result in results:
                print(f"  Categoria: {result['_id']['category']}, Status: {result['_id']['status']}, Quantidade: {result['count']}")
            return results
        else:
            print("Nenhuma contagem por categoria e status encontrada.")
            return []
    except Exception as e:
        print(f"ERRO ao contar bens por categoria e status: {e}")
        return []

# Executa as funções de agregação
if bens_collection is not None:
    contar_bens_por_localizacao(bens_collection)
    contar_bens_por_categoria_e_status(bens_collection)

In [ ]:
# Célula 12: Análise Exploratória de Dados com Pandas (Item 6)

# Item 6: Faça uma análise exploratória dos dados via pandas e apresente.

def analise_exploratoria(collection):
    """
    Realiza uma análise exploratória dos dados da coleção usando Pandas.
    Item 6a: Distribuição de frequência de um documento e campo.
    Item 6b: Visão geral dos tipos e métricas (avg, std, quartis, etc.) de um documento.
    """
    print("\n--- ANÁLISE EXPLORATÓRIA DE DADOS COM PANDAS ---")
    try:
        # Busca todos os documentos para a análise exploratória (excluindo o _id).
        data = list(collection.find({}, {"_id": 0}))
        if not data:
            print("Nenhum dado para análise exploratória.")
            return pd.DataFrame()

        df = pd.DataFrame(data)

        # 6a. Distribuição de frequência de um documento e campo (ex: 'location')
        print("\nVisão Geral do DataFrame (Head):")
        print(df.head().to_string())

        print("\nInformações do DataFrame (Tipos de Dados e Não-Nulos):")
        df.info()

        print("\n--- 6a. Distribuição de Frequência da Localização (df['location'].value_counts()) ---")
        location_counts = df['location'].value_counts()
        print(location_counts)

        # 6b. Visão geral dos tipos e métricas (avg, std, quartis, entre outros) de um documento.
        # Para campos numéricos, 'describe()' funciona diretamente.
        print("\n--- 6b. Métricas Descritivas de Campos Numéricos (se existirem) ---")
        numeric_cols = df.select_dtypes(include=['number']).columns
        if not numeric_cols.empty:
            print(df[numeric_cols].describe().to_string())
        else:
            print("Não há colunas numéricas diretas no DataFrame para 'describe()'.")
            print("\nExemplo de métricas para contagem de bens por categoria (contagem de ocorrências):")
            category_counts = df['category'].value_counts()
            print(category_counts.describe().to_string())


        return df
    except Exception as e:
        print(f"ERRO durante a análise exploratória: {e}")
        return pd.DataFrame()

# Executa a análise exploratória
if bens_collection is not None:
    df_full_data_for_analysis = analise_exploratoria(bens_collection)

In [ ]:
# Célula 13: Geração de Gráficos (Item 7)

# Item 7: Monte 2 gráficos diferentes e apresente os dados que achar pertinente.

def gerar_graficos(df):
    """
    Gera dois gráficos diferentes a partir do DataFrame para visualização dos dados.
    """
    if df.empty:
        print("AVISO: DataFrame vazio, não é possível gerar gráficos.")
        return

    print("\n--- GERANDO RELATÓRIOS VISUAIS ---")

    # Gráfico 1: Quantidade de Bens por Localização (Gráfico de Barras)
    plt.figure(figsize=(12, 7))
    sns.countplot(data=df, y='location', order=df['location'].value_counts().index, hue='location', legend=False, palette='viridis')
    plt.title('Distribuição de Bens por Localização', fontsize=16)
    plt.xlabel('Quantidade de Bens', fontsize=12)
    plt.ylabel('Localização', fontsize=12)
    plt.grid(axis='x', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.savefig('bens_por_localizacao.png') # Salva o gráfico como imagem
    print("Gráfico 'bens_por_localizacao.png' gerado e salvo com sucesso.")
    plt.show() # Exibe o gráfico no Jupyter

    # Gráfico 2: Distribuição de Bens por Categoria (Gráfico de Pizza)
    # Selecionar as 10 categorias mais comuns 
    top_categories = df['category'].value_counts().nlargest(10)
    if not top_categories.empty:
        plt.figure(figsize=(10, 10))
        colors = plt.get_cmap('Pastel1')(range(len(top_categories)))
        plt.pie(top_categories, labels=top_categories.index, autopct='%1.1f%%', startangle=90, colors=colors, pctdistance=0.65) # 'cmap' foi removido
        plt.title('Distribuição Percentual de Bens \npor Categoria (Top 10)\n', fontsize=15, fontweight='bold')
        plt.axis('equal') # Garante que o gráfico de pizza seja um círculo.
        plt.tight_layout()
        plt.savefig('bens_por_categoria_pizza.png') # Salva o gráfico como imagem
        print("Gráfico 'bens_por_categoria_pizza.png' gerado e salvo com sucesso.")
        plt.show() # Exibe o gráfico no Jupyter
    else:
        print("AVISO: Não há categorias suficientes para gerar o gráfico de pizza.")



# Executa a geração de gráficos, se houver dados para análise
# Certifique-se de que df_full_data_for_analysis esteja disponível e não vazio (definido na Célula 12)
if bens_collection is not None:
    try:
        # Tenta reobter o DataFrame caso o notebook tenha sido reiniciado ou a célula 12 não rodou
        # Em uma execução "Run All", df_full_data_for_analysis já estará populado.
        data_for_graphs = list(bens_collection.find({}, {"_id": 0}))
        if data_for_graphs:
            df_for_graphs = pd.DataFrame(data_for_graphs)
            gerar_graficos(df_for_graphs)
        else:
            print("\nNão foi possível gerar gráficos: Nenhuma dado disponível no MongoDB para análise.")
    except NameError:
        print("\nErro: 'df_full_data_for_analysis' não foi definido. Certifique-se de executar a Célula 12 (Análise Exploratória) primeiro.")
    except Exception as e:
        print(f"\nErro ao tentar gerar gráficos: {e}")

In [ ]:
# Célula 14: Conclusão e Próximos Passos

print("\n--- DEMONSTRAÇÃO DO PROJETO FINAL CONCLUÍDA ---")
print("Todas as funcionalidades obrigatórias do trabalho foram demonstradas:")
print("- Conexão e importação de dados para MongoDB.")
print("- Operações CRUD (Inserção, Edição, Busca, Exclusão) de documentos.")
print("- Buscas com operadores e funções de agregação.")
print("- Análise exploratória de dados com Pandas.")
print("- Geração de relatórios visuais (gráficos).")

print("\nObrigado(a)!")